# Project 3 : Web APIs & Classification

### Importing Libraries

In [380]:
import requests
import time
import pandas as pd
import numpy as np
import csv
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
import seaborn as sns

## Web Scrapping

In [36]:
url_politics = 'https://www.reddit.com/r/politics.json'

In [55]:
url_sports = 'https://www.reddit.com/r/sports.json'

In [56]:
headers = {'User-agent': 'Naoufal'}

In [38]:
res_politics = requests.get(url_politics)

In [57]:
res_sports = requests.get(url_sports)

In [58]:
print(res_politics)
print(res_sports)

<Response [200]>
<Response [200]>


In [59]:
res_politics_content = res_politics.content

In [60]:
res_sports_content = res_sports.content

In [64]:
res_politics_json = res_politics.json()

In [65]:
res_sports_json = res_sports.json()

In [68]:
res_politics_json.keys()

dict_keys(['kind', 'data'])

In [69]:
res_sports_json.keys()

dict_keys(['kind', 'data'])

In [70]:
res_politics_json['data'].keys()

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])

In [71]:
res_sports_json['data'].keys()

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])

In [72]:
[post['data']['name'] for post in res_politics_json['data']['children']]

['t3_di9dad',
 't3_digrkk',
 't3_dieoev',
 't3_difal4',
 't3_difeo0',
 't3_did11u',
 't3_dicaua',
 't3_die835',
 't3_dibasd',
 't3_dia959',
 't3_dib583',
 't3_dihnvu',
 't3_didwpl',
 't3_diexor',
 't3_dib7h8',
 't3_difnkx',
 't3_di9u7q',
 't3_did94d',
 't3_di9bhy',
 't3_dibgdo',
 't3_di879i',
 't3_di8hku',
 't3_di7ph3',
 't3_diavxs',
 't3_die7ak',
 't3_digsdd',
 't3_difdac']

In [73]:
[post['data']['name'] for post in res_sports_json['data']['children']]

['t3_di8fwt',
 't3_di3qq7',
 't3_difamf',
 't3_dicjp2',
 't3_dicv47',
 't3_dihy49',
 't3_dhtfzp',
 't3_di6pgw',
 't3_difruh',
 't3_di0xcu',
 't3_diaj27',
 't3_diixes',
 't3_di9noe',
 't3_di855c',
 't3_dhz76e',
 't3_di6fuc',
 't3_diiex8',
 't3_dhi4sd',
 't3_dibb2o',
 't3_di1zbz',
 't3_dhzllc',
 't3_dhwqgn',
 't3_dh9lgu',
 't3_dhzg1d',
 't3_dhocg5']

In [74]:
res_politics_json['data']['after']

't3_difdac'

In [75]:
res_sports_json['data']['after']

't3_dhocg5'

In [77]:
politics_params = {'after': res_politics_json['data']['after']}

In [78]:
sports_params = {'after': res_sports_json['data']['after']}

In [103]:
jsons_politics =[]
after = None
for i in range (40):
    print(i)
    if after ==None:
        politics_params = {}
    else:
        politics_params = {'after': after}
        
    url_politics = 'https://www.reddit.com/r/politics.json'
    res_politics_2 = requests.get(url_politics, params=politics_params, headers=headers)
    
    if res_politics_2.status_code ==200:
        res_politics_json = res_politics_2.json()
        jsons_politics.extend(res_politics_json['data']['children'])
        after = res_politics_json['data']['after']
        
    else:
        print(res_politics_2.status_code)
        break
        
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [100]:
jsons_sports =[]
after = None
for i in range (40):
    print(i)
    if after ==None:
        sports_params = {}
    else:
        sports_params = {'after': after}
        
    url_sports = 'https://www.reddit.com/r/sports.json'
    res_sports_2 = requests.get(url_sports, params=sports_params, headers=headers)
    
    if res_sports_2.status_code ==200:
        res_sports_json = res_sports_2.json()
        jsons_sports.extend(res_sports_json['data']['children'])
        after = res_sports_json['data']['after']
        
    else:
        print(res_psports_2.status_code)
        break
        
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [106]:
len(jsons_politics)

983

In [107]:
len(set([p['data']['name'] for p in jsons_politics]))

906

In [101]:
len(jsons_sports)

978

In [102]:
len(set([p['data']['name'] for p in jsons_sports]))

339

In [110]:
jsons_politics[0].keys()

dict_keys(['kind', 'data'])

In [111]:
jsons_sports[0].keys()

dict_keys(['kind', 'data'])

## Converting Dict to DataFrame 

In [317]:
df_politics = pd.DataFrame(jsons_politics)

In [318]:
df_politics.shape

(983, 2)

In [319]:
df_politics.head()

,kind,data
0,t3,"{'approved_at_utc': None, 'subreddit': 'politi..."
1,t3,"{'approved_at_utc': None, 'subreddit': 'politi..."
2,t3,"{'approved_at_utc': None, 'subreddit': 'politi..."
3,t3,"{'approved_at_utc': None, 'subreddit': 'politi..."
4,t3,"{'approved_at_utc': None, 'subreddit': 'politi..."


In [320]:
df_politics = pd.DataFrame(df_politics['data'].values.tolist(), index=df_politics.index)

In [321]:
df_politics.shape

(983, 109)

In [322]:
df_sports = pd.DataFrame(jsons_sports)

In [323]:
df_sports.head()

,kind,data
0,t3,"{'approved_at_utc': None, 'subreddit': 'sports..."
1,t3,"{'approved_at_utc': None, 'subreddit': 'sports..."
2,t3,"{'approved_at_utc': None, 'subreddit': 'sports..."
3,t3,"{'approved_at_utc': None, 'subreddit': 'sports..."
4,t3,"{'approved_at_utc': None, 'subreddit': 'sports..."


In [324]:
df_sports = pd.DataFrame(df_sports['data'].values.tolist(), index=df_sports.index)

In [325]:
df_sports.shape

(978, 105)

In [326]:
df.shape

(1313, 4255)

## Merging & Cleaning DataFrames 

In [327]:
df_politics['label']=0

In [328]:
df_sports['label']=1

In [329]:
df = pd.concat((df_politics, df_sports))

/Users/naoufal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [330]:
df['title'].head(10)

0    We are leaders at NALEO Educational Fund. We’r...
1    Discussion Thread: Fourth Democratic President...
2    'Arrest and Detain Giuliani Right Now,' Say Cr...
3    Pence refuses House request to provide documen...
4    Erdogan tells Trump that Turkey will 'never de...
5    RNC Chairwoman Decries Nepotism In Politics — ...
6    Giuliani says he won't comply with a congressi...
7    The floodgates are opening as Trump officials ...
8    Giuliani pressed Trump to eject Muslim cleric ...
9    New Poll Suggests Sen. Susan Collins’ Support ...
Name: title, dtype: object

In [132]:
df.to_csv('df_politics_sports.csv', index=False)

In [331]:
df.shape

(1961, 110)

In [332]:
df['combined']=df.apply(lambda x:'%s_%s' % (x['title'],x['selftext']),axis=1)

In [333]:
df['combined'] = df['combined'].str.lower()

In [334]:
df[['label', 'title', 'selftext', 'combined']]

,label,title,selftext,combined
0,0,We are leaders at NALEO Educational Fund. We’r...,"Hi Reddit! We are Arturo Vargas, CEO and Lizet...",we are leaders at naleo educational fund. we’r...
1,0,Discussion Thread: Fourth Democratic President...,"This is the ""post-debate"" thread for the fourt...",discussion thread: fourth democratic president...
2,0,"'Arrest and Detain Giuliani Right Now,' Say Cr...",,"'arrest and detain giuliani right now,' say cr..."
3,0,Pence refuses House request to provide documen...,,pence refuses house request to provide documen...
4,0,Erdogan tells Trump that Turkey will 'never de...,,erdogan tells trump that turkey will 'never de...
...,...,...,...,...
973,1,Unbelievable catch by Jonathan Carter in the CPL,,unbelievable catch by jonathan carter in the cpl_
974,1,High school running back gets flipped over and...,,high school running back gets flipped over and...
975,1,Youth sports officials and referees quitting a...,,youth sports officials and referees quitting a...
976,1,Essex's captain Simon Harmer lifts the Vitalit...,,essex's captain simon harmer lifts the vitalit...


## Model prep

In [335]:

X = df['combined']
y = df['label']

In [336]:
y.value_counts()

0    983
1    978
Name: label, dtype: int64

In [337]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    stratify=y,
                                                    random_state=42)

In [338]:
cvec = CountVectorizer()

In [340]:
cvec.fit(X_train)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [341]:
X_train = cvec.transform(X_train)

In [342]:
X_train.shape

(1313, 4255)

In [343]:
X_train_df = pd.DataFrame(X_train.toarray(),
                          columns=cvec.get_feature_names())
X_train_df

,000,000m,00_,04,041lbs,04_,0_,10,1000,1000_,...,yu,yun,zdenrk,zealand,zealands,zhiyong,zone_,zuckerberg,zuckerberg_,zuerlein
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1311,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [344]:
X_train[:5]

<5x4255 sparse matrix of type '<class 'numpy.int64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [345]:
X_test = cvec.transform(X_test)
X_test_df = pd.DataFrame(X_test.toarray(),
                         columns=cvec.get_feature_names())

In [346]:

print(stop_words.ENGLISH_STOP_WORDS)

frozenset({'keep', 'latter', 'herself', 'most', 'nevertheless', 'please', 'further', 'per', 'whereafter', 'none', 'next', 'this', 'twelve', 'until', 'amount', 'becomes', 'once', 'mine', 'that', 'any', 'eg', 'back', 'less', 'now', 'somewhere', 'namely', 'via', 'find', 'myself', 'below', 'formerly', 'go', 'meanwhile', 'others', 'might', 'detail', 'whereby', 'who', 'interest', 'seems', 'system', 'on', 'were', 'empty', 'into', 'front', 'my', 'moreover', 'see', 'amongst', 'top', 'besides', 'is', 'through', 'hereafter', 'hundred', 'still', 'almost', 'along', 'due', 'too', 'will', 'whose', 'are', 'beyond', 'or', 'throughout', 'also', 'get', 'co', 'whence', 'sincere', 'last', 'yourselves', 'here', 'twenty', 'then', 'except', 'whereas', 'hereupon', 'least', 'we', 'why', 'before', 'beforehand', 'her', 'hers', 'has', 'ten', 'four', 'noone', 'serious', 'whither', 'would', 'him', 'un', 'two', 'which', 'by', 'etc', 'neither', 'nowhere', 'over', 'such', 'whom', 'ours', 'of', 'somehow', 'afterwards', 

In [347]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    stratify=y,
                                                    random_state=42)

In [348]:
y_test.value_counts(normalize=True)

0    0.501543
1    0.498457
Name: label, dtype: float64

In [349]:
pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr', LogisticRegression())
])

In [350]:
pipe_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [1, 2],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2)]
}

In [351]:
gs = GridSearchCV(pipe, 
                  param_grid=pipe_params, 
                  cv=3) 

In [353]:
gs.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'cvec__max_features': [2500, 3000, 3500], 'cvec__min_df': [1, 2], 'cvec__max_df': [0.9, 0.95], 'cvec__ngram_range': [(1, 1), (1, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [354]:
print(gs.best_score_)

0.964965727341965


In [355]:
gs_model = gs.best_estimator_

In [356]:
gs_model.score(X_train, y_train)

0.9992383853769993

In [357]:
gs_model.score(X_test, y_test)

0.9814814814814815

In [358]:
tvec = TfidfVectorizer()

In [360]:
df = pd.DataFrame(tvec.fit_transform(X_train).toarray(),
                  columns=tvec.get_feature_names())
df.head()

,000,000m,00_,04,041lbs,04_,0_,10,1000,1000_,...,yu,yun,zdenrk,zealand,zealands,zhiyong,zone_,zuckerberg,zuckerberg_,zuerlein
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.401602,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [361]:
X_train = tvec.fit_transform(X_train)

X_test = tvec.transform(X_test)

In [363]:

lr = LogisticRegression()


lr.fit(X_train, y_train)


print(f'Training Score: {lr.score(X_train, y_train)}')
print(f'Testing Score: {lr.score(X_test, y_test)}')

Training Score: 0.9977151561309977
Testing Score: 0.9768518518518519


In [375]:

print(f'Logistic Regression Coefficient: {lr.coef_}')

Logistic Regression Coefficient: [[-0.29949879  0.17422428  0.13554683 ... -0.39221971 -0.10265872
   0.12419944]]


In [378]:
np.exp(lr.coef_)

array([[0.74118962, 1.19032251, 1.14516283, ..., 0.67555567, 0.90243491,
        1.13224166]])